# Percentile Analysis

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm

In [2]:
df_dataset = pd.read_csv('../data/processed/processed_daysahead3_R000.csv', index_col=0)
df_dataset.index = pd.to_datetime(df_dataset.index)
df_dataset.head()

,sigma,Vp_pred,Vp_obs
2010-01-08 06:00:00,68.801098,317.327273,301.767500
2010-01-08 12:00:00,65.147504,328.554545,286.885000
2010-01-08 18:00:00,68.857374,320.569697,283.535000
2010-01-09 00:00:00,64.175179,312.584848,289.951667
2010-01-09 06:00:00,57.276227,304.600000,294.783333


In [3]:
inside = {
    50: [],
    75: [],
    90: [],
    95: [],
}

for _, row in df_dataset.iterrows():
    for percentile, records in inside.items():
        left, right = norm(loc=row.Vp_pred, scale=row.sigma).interval(percentile/100)
        records.append(bool(row.Vp_obs > left and row.Vp_obs < right))

In [7]:
df_rows = []

for percentile, records in inside.items():
    df_rows.append((percentile, 100*np.mean(records), 100*np.mean(records) - percentile, len(records)))

pd.set_option('display.float_format', '{:.1f}'.format)

df_output = pd.DataFrame(df_rows, columns=['Percentile', 'RateObsIncluded', 'Difference', 'SampleCount'])
df_output

,Percentile,RateObsIncluded,Difference,SampleCount
0,50,49.5,-0.5,16025
1,75,72.9,-2.1,16025
2,90,88.4,-1.6,16025
3,95,93.9,-1.1,16025
